# Entrenamiento Detección

## Instalaciones  



In [ ]:
import sys

IN_COLAB = 'google.colab' in sys.modules

if IN_COLAB:
    !pip install  ultralytics
    from google.colab import drive
    drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Imports Imports

In [ ]:
import os
import ultralytics
ultralytics.checks()
import cv2
import yaml
from ultralytics import YOLO
from datetime import datetime
import torch
import numpy as np
from pathlib import Path

Ultralytics 8.3.150 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
Setup complete ✅ (2 CPUs, 12.7 GB RAM, 42.1/112.6 GB disk)


## Paths

#### Función poner el path correcto

In [ ]:
def get_paths(name_bussines, name_project, IN_COLAB=False, sub_yaml=None,
              model_name=None, video_name=None, output_video_name=None):
    # Definir ruta base según entorno
    if IN_COLAB:
        base_dir = os.path.join('/content/drive/MyDrive', name_bussines, name_project)
    else:
        base_dir = os.path.join('/home/usuario/proyectos', name_project)

    os.chdir(base_dir)

    # Rutas base comunes
    paths = {
        'BASE_DIR': base_dir,
        'IMAGES_DIR': os.path.join(base_dir, 'dataset', 'images'),
        'MODEL_DIR': os.path.join(base_dir, 'runs/train'),
    }

    # Si se proporciona sub_yaml
    if sub_yaml is not None:
        sub_path = os.path.join(base_dir, sub_yaml)
        paths['DATA_YAML'] = os.path.join(sub_path, 'data.yaml')
        paths['VALID_IMAGES_DIR'] = os.path.join(sub_path, 'valid', 'images')

        if model_name is None:
            model_stub = sub_yaml.split("-")[0].split("_")[-1].lower()
            number = sub_yaml.split("-")[1] if '-' in sub_yaml else '1'
            model_name = f"{model_stub}_{number}"
        if video_name is None:
            video_name = 'camera_2_video.mp4'

        if output_video_name is None:
            output_video_name = f'output_{model_name}-detectado.mp4'

        # Agregar rutas adicionales inferidas
        paths['MODEL_WEIGHTS'] = os.path.join(base_dir, 'runs', 'train', model_name, 'weights', 'best.pt')
        paths['VIDEO_PATH'] = os.path.join(base_dir, video_name)
        paths['OUTPUT_VIDEO_PATH'] = os.path.join(base_dir, output_video_name)
        paths['OUTPUT_DIR'] = os.path.join(base_dir, f'output_{model_name}-img')

    return paths

#### Debemos poner el path correcto

In [ ]:
#  '/content/drive/MyDrive/Smartech/Helados'
#  '/content/drive/MyDrive/name_bussines/name_project' Aquí estan todos los archivos como video y este archivo ipynb

name_bussines = 'Smartech' #
name_project = 'MinaRumas'
model_name="ruma_3"              # modelo personalizado, verificar
video_name="camera_2_video.mp4"  # video de entrada personalizado
output_video_name="output_video.mp4"   # video de salida personalizado

paths = get_paths(name_bussines, name_project, IN_COLAB = IN_COLAB) # cargamos los path

### Elegimos yolo11s (segmentacion)

In [ ]:
model = YOLO('yolo11m-seg.pt')

# Credenciales de Roboflow

#### Instalación de roboflow

In [ ]:
!pip install roboflow

#### Credenciales

In [ ]:
from roboflow import Roboflow

rf = Roboflow(api_key="kLnigLCsXTY0ACh4bZuT")
project = rf.workspace("smartech-cekkw").project("alma-ruma-segmentation")
version = project.version(3)
dataset = version.download("yolov11")



loading Roboflow workspace...
loading Roboflow project...


#### Paths

In [ ]:
sub_yaml = f"{project.name}-{version.version}"

paths = get_paths(
    name_bussines,
    name_project,
    IN_COLAB=IN_COLAB,
    sub_yaml=sub_yaml,
    model_name=model_name,
    video_name=video_name,
    output_video_name=output_video_name
)

# Imprimir todos los paths
for key, value in paths.items():
    print(f"{key}: {value}")

BASE_DIR: /content/drive/MyDrive/Smartech/MinaRumas
IMAGES_DIR: /content/drive/MyDrive/Smartech/MinaRumas/dataset/images
MODEL_DIR: /content/drive/MyDrive/Smartech/MinaRumas/runs/train
DATA_YAML: /content/drive/MyDrive/Smartech/MinaRumas/Alma-Ruma-Segmentation-3/data.yaml
VALID_IMAGES_DIR: /content/drive/MyDrive/Smartech/MinaRumas/Alma-Ruma-Segmentation-3/valid/images
MODEL_WEIGHTS: /content/drive/MyDrive/Smartech/MinaRumas/runs/train/ruma_3/weights/best.pt
VIDEO_PATH: /content/drive/MyDrive/Smartech/MinaRumas/camera_2_video.mp4
OUTPUT_VIDEO_PATH: /content/drive/MyDrive/Smartech/MinaRumas/output_video.mp4
OUTPUT_DIR: /content/drive/MyDrive/Smartech/MinaRumas/output_ruma_3-img


## Entrenamiento

In [9]:
device =  0  #  Usar CPU= "cpu", GPU=  0 GPU
batch = 8 # Batch size
imgsz = 1024 # Tamaño de imagen
seed = 22 # Semilla para reproducibilidad
cache = True  # Cache en RAM (más rápido)
epochs = 250  # Épocas de entrenamiento

model.train(# Entrenar modelo
    data=paths['DATA_YAML'],
    epochs=epochs,
    device=device,
    batch=batch,
    imgsz = imgsz,
    seed = seed,
    cache = cache,
    exist_ok = True,                      # Sobrescribe carpeta si ya existe
    project = paths['MODEL_DIR'],      # Carpeta base de resultados
    name = model_name         # Nombre del experimento
)

Ultralytics 8.3.150 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=8, bgr=0.0, box=7.5, cache=True, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/drive/MyDrive/Smartech/MinaRumas/Alma-Ruma-Segmentation-3/data.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=250, erasing=0.4, exist_ok=True, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=1024, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolo11m-seg.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=ruma_3, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=100, p

train: Scanning /content/drive/MyDrive/Smartech/MinaRumas/Alma-Ruma-Segmentation-3/train/labels.cache... 72 images, 0 backgrounds, 0 corrupt: 100%|██████████| 72/72 [00:00<?, ?it/s]


WARNING ⚠️ cache='ram' may produce non-deterministic training results. Consider cache='disk' as a deterministic alternative if your disk space allows.


train: Caching images (0.1GB RAM): 100%|██████████| 72/72 [00:02<00:00, 34.56it/s]


albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, method='weighted_average', num_output_channels=3), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))
val: Fast image access ✅ (ping: 0.4±0.1 ms, read: 91.7±30.2 MB/s, size: 348.7 KB)


...

                   all          7        147      0.875       0.85      0.905      0.832      0.882      0.857      0.922      0.803



250 epochs completed in 0.879 hours.
Optimizer stripped from /content/drive/MyDrive/Smartech/MinaRumas/runs/train/ruma_3/weights/last.pt, 45.2MB
Optimizer stripped from /content/drive/MyDrive/Smartech/MinaRumas/runs/train/ruma_3/weights/best.pt, 45.2MB

Validating /content/drive/MyDrive/Smartech/MinaRumas/runs/train/ruma_3/weights/best.pt...
Ultralytics 8.3.150 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
YOLO11m-seg summary (fused): 138 layers, 22,336,083 parameters, 0 gradients, 123.0 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Mask(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:00<00:00,  3.65it/s]


                   all          7        147      0.824      0.932      0.918      0.845       0.83      0.939      0.932       0.82
Speed: 0.4ms preprocess, 26.9ms inference, 0.0ms loss, 1.5ms postprocess per image
Results saved to /content/drive/MyDrive/Smartech/MinaRumas/runs/train/ruma_3


ultralytics.utils.metrics.SegmentMetrics object with attributes:

ap_class_index: array([0])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x79d24d5ad210>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)', 'Precision-Recall(M)', 'F1-Confidence(M)', 'Precision-Confidence(M)', 'Recall-Confidence(M)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041, 

# Pruebas con modelo entrenado

## Imagenes

In [ ]:
from ultralytics import YOLO
import os
import cv2

# Cargar modelo
model = YOLO(paths['MODEL_WEIGHTS'])

image_dir = paths['VALID_IMAGES_DIR']
output_dir = paths['OUTPUT_DIR']

os.makedirs(output_dir, exist_ok=True)

# Obtener lista de imágenes
image_files = [f for f in os.listdir(image_dir) if f.endswith(('.jpg', '.png', '.jpeg'))]

for filename in image_files:
    image_path = os.path.join(image_dir, filename)

    # Ejecutar predicción
    results = model.predict(source=image_path, imgsz=1024, conf=0.25, save=False, verbose=False, task='segment')
    result = results[0]  # solo una imagen

    print(f"\n📷 Imagen: {filename}")

    class_ids = result.boxes.cls.int().tolist()
    class_names = [model.names[c] for c in class_ids]
    boxes = result.boxes.xyxy.tolist()

    for i, box in enumerate(boxes):
        print(f"📦 Box {i+1}: {box} - Clase: {class_names[i]}")

    # Dibujar resultados y guardar imagen
    annotated_frame = result.plot()  # Incluye máscaras y cajas
    save_path = os.path.join(output_dir, filename)
    cv2.imwrite(save_path, annotated_frame)
    print(f"✅ Guardado en: {save_path}")
    print("ok")

## Video Prueba

In [11]:
video_path = paths['VIDEO_PATH']
model_path =paths['MODEL_WEIGHTS']
output_path = paths['OUTPUT_VIDEO_PATH']
# video_path = 'camera_2_video.mp4'  # Ruta real de tu video
# model_path = '/content/drive/MyDrive/Smartech/Helados/runs/train/freh_1/weights/best.pt'  # Ruta real del modelo
#output_path = 'output_detectado.mp4'
start_time_sec = 0
end_time_sec = 32

In [12]:
# Inicializar video
cap = cv2.VideoCapture(video_path)
fps = cap.get(cv2.CAP_PROP_FPS)
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

start_frame = int(start_time_sec * fps)
end_frame = int(end_time_sec * fps)

# Preparar VideoWriter
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))

# Cargar modelo
model = YOLO(model_path)

# Función para dibujar los resultados de detección
def draw_detections(frame, results):
    for result in results:
        boxes = result.boxes
        for i in range(len(boxes)):
            box = boxes.xyxy[i].cpu().numpy().astype(int)  # Coordenadas [x1, y1, x2, y2]
            cls_id = int(boxes.cls[i].item())
            conf = boxes.conf[i].item()

            # Dibujar caja
            cv2.rectangle(frame, (box[0], box[1]), (box[2], box[3]), (0, 255, 0), 2)
            label = f'{model.names[cls_id]} {conf:.2f}'
            cv2.putText(frame, label, (box[0], box[1]-10), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)
    return frame

# Procesamiento
frame_count = 0
with torch.no_grad():
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret or frame_count > end_frame:
            break
        if frame_count >= start_frame:
            if frame_count % 100 == 0:
                print(f"Procesando frame: {frame_count}")
            results = model.predict(frame, verbose=False)
            frame = draw_detections(frame, results)
            out.write(frame)
        frame_count += 1

# Liberar recursos
cap.release()
out.release()
print("✅ Video procesado guardado en:", output_path)


✅ Video procesado guardado en: /content/drive/MyDrive/Smartech/MinaRumas/output_video.mp4
